In [ ]:
from matplotlib import pyplot as plot
from scipy.stats import gamma
from itertools import product
from tqdm import tqdm
import numpy

%matplotlib inline

In [ ]:
# Standard differentially-private schemes

def gaussian_noise(location, epsilon):
    return location + numpy.random.normal(loc = [0, 0], scale = 1 / epsilon )

def laplacian_noise(location, epsilon):
    return location + numpy.random.laplace(loc = [0, 0], scale = 1 / epsilon )

def geo_noise(location, epsilon):
    theta, r = numpy.random.uniform(0, 2 * numpy.pi), gamma.ppf( numpy.random.uniform(0, 1), 2, scale = (1 / epsilon) )
    x, y = r * numpy.cos(theta), r * numpy.sin(theta)
    return location + numpy.array([x, y])

# An extension of k-anonymity
def cloaking(location, epsilon):
    x, y = location
    return ( x // 5, y // 5 )

In [ ]:
privacy = {
    "Gaussian": gaussian_noise,
    "Laplacian": laplacian_noise,
    "Geo-Indistinguishability": geo_noise,
    "Cloaking": cloaking
}

In [ ]:
def distance(location, center):
    x, y = location
    a, b = center
    return numpy.sqrt( (x - a)**2 + (y - b)**2 )

# We assume users can be modeled as having some distribution over locations, 
# which is centered at some "home" but is non-deterministic (ie., noisy)

class user:
    def __init__(self, server):
        self.server = server
        self.location_distr = self.centered_distribution()
        self.location_distr = self.location_distr / numpy.sum(self.location_distr)
        
    def centered_distribution(self):
        center  = self.server.space[ numpy.random.randint(0, len(self.server.space)) ]
        initial = [ numpy.random.uniform(0, 2) / (distance(L, center) + 1) for L in self.server.space ]
        return numpy.round( numpy.array( initial ) / numpy.sum( initial ), decimals = 2 )
        
    def sample(self, epochs):
        index = numpy.random.choice( len(self.server.space), size = epochs, p = self.location_distr )
        return [ self.server.space[i] for i in index ]

In [ ]:
# A visual of a user's distribution
numpy.random.seed(0)

u = server(length = 25, count = 1, epochs = 500)
plot.scatter(* zip(* u.space), s = u.users[0].location_distr * 100, c = "black")
plot.show()

In [ ]:
# The semi-honest server coordinates between users, has knowledge of the entire space,
# and can make predications to a user's distribution given a number of samples

class server:
    def __init__(self, length = 75, count = 1000, epochs = 250):
        self.space = product(range(length), repeat = 2 )
        self.space = list( map(tuple, self.space) )
        
        self.users = [ user(self) for i in range(count) ]
        self.samples = [ u.sample(epochs) for u in self.users ]

    def clean(self, location):
        return tuple(numpy.round(location))

    def noisy_samples(self, noise, epsilon):
        return [ [ self.clean(noise(s, epsilon)) for s in samples ] for samples in self.samples ]
    
    def noisy_estimate(self, samples, epoch, discretized = False):
        if not discretized:
            best_estimate = [ samples[: epoch].count(s) for s in self.space ]
        else:
            best_estimate = [ samples[: epoch].count((x // 5, y // 5)) for x, y in self.space ]
            
        best_estimate = numpy.array(best_estimate) / sum(best_estimate)    
        return best_estimate
    
    def bhattacharyya(self, estimate, true):
        return numpy.sum(numpy.sqrt(estimate * true))
    
    def average_similarity(self, samples, epoch):
        estimates = [ self.noisy_estimate(sample, epoch) for sample in samples ]
        true_vals = [ user.location_distr for user in self.users ]
        
        similarities = [ self.bhattacharyya(e, t) for e, t in zip(estimates, true_vals) ]
        return numpy.mean(similarities)
    
    def epoch_plot(self, noise, epsilon, name = ""):
        noisy_sampling = self.noisy_samples(noise, epsilon)
        
        epoch_count  = numpy.arange(1, len(self.samples[0]))
        similarities = [ self.average_similarity(noisy_sampling, x) for x in epoch_count ]
        
        plot.plot(epoch_count, similarities, label = name)
        
    def compare_noises(self, epsilon):
        for noise in privacy:
            self.epoch_plot( privacy[noise], epsilon, noise )

        plot.legend()
        plot.show()

In [ ]:
USGS = server(length = 25, count = 100, epochs = 500)

In [ ]:
USGS.compare_noises(0.02)